---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [2]:
import networkx as nx

# This line must be commented out when submitting to the autograder
!head email_network.txt

#Sender	Recipient	time
1	2	1262454010
1	3	1262454010
1	4	1262454010
1	5	1262454010
1	6	1262454010
1	7	1262454010
1	8	1262454010
1	9	1262454010
1	10	1262454010


### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [3]:
import pandas as pd
def strs():
    return [str(x) for x in range(1,168)]
def answer_one():
    df = pd.read_csv('email_network.txt', sep='\t')
    G = nx.from_pandas_dataframe(df, '#Sender', 'Recipient', create_using=nx.MultiDiGraph())
    #G = nx.read_edgelist('email_network.txt', delimiter='\t', data=[('time', int)], create_using=nx.MultiDiGraph())
    mapping=dict(zip(G.nodes(), strs()))
    G = nx.relabel_nodes(G, mapping)
    return G
answer_one()

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [4]:
def answer_two():
        
    G = answer_one()
    employees = nx.number_of_nodes(G)
    emails = nx.number_of_edges(G)
    
    return tuple((employees, emails))
answer_two()

(167, 82927)

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [5]:
def answer_three():
    G = answer_one()
    
    '''Since our graph is a directed graph, 
    and the communication channel is one-way
    we use the is_strongly_connected() function''' 
    part1 = nx.is_strongly_connected(G)
    
    '''Since our graph is a directed graph, 
    and the communication channel is two-way
    we first need to convert to undirected''' 
    part2 = nx.is_connected(G.to_undirected())
    return tuple((part1, part2))
answer_three()

(False, True)

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [6]:
def answer_four():
    G = answer_one()
    largest_cc = len(max(nx.weakly_connected_components(G), key=len))
    return largest_cc
answer_four()

167

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [7]:
def answer_five():    
    G = answer_one()
    largest_cc = len(max(nx.strongly_connected_components(G), key=len))     
    return largest_cc
answer_five()

126

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [8]:
def answer_six():
    G = answer_one()
    
    #Looked up the docs
    G_sc = max(nx.strongly_connected_component_subgraphs(G), key=len)
    
    return G_sc
answer_six()

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [9]:
def answer_seven():
    G_sc = answer_six()
    
    return nx.average_shortest_path_length(G_sc)
answer_seven()

1.6461587301587302

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [10]:
def answer_eight():
    G_sc = answer_six()

    return nx.diameter(G_sc)
answer_eight()

3

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [11]:
def answer_nine():
    G_sc = answer_six()
    
    '''By definition the question asks us
    to find the Periphery of the graph'''
    
    return set(nx.periphery(G_sc))
answer_nine()

{'129', '134', '97'}

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [12]:
def answer_ten():
        
    G_sc = answer_six()
    
    '''By definition the question asks us
    to find the Center of the graph'''
    
    return set(nx.center(G_sc))
answer_ten()

{'38'}

### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [30]:
def answer_eleven():
        
    G_sc = answer_six()
    diameter = answer_eight()
    
    #I initialize the variables
    answer_node = None
    maximum_number = 0
    
    #Loop through the set of peripheries
    for node in nx.periphery(G_sc):
        number = 0
        # Get shortest path lengths equal to diameter
        for item, length in nx.shortest_path_length(G_sc, node).items():
            if length==diameter:
                number = number+1
        #Set the new values
        if number > maximum_number:
            maximum_number = number
            answer_node = node
    #return max(nx.shortest_path_length(G_sc, '1').items(), key=lambda e: e[1])
    return (answer_node, maximum_number)
    
answer_eleven()

('97', 63)

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [39]:
def answer_twelve():
    G_sc = answer_six()    
    #We have:
    #source = center
    #target = node from the previous question
    return len(nx.minimum_node_cut(G_sc, nx.center(G_sc)[0], answer_eleven()[0] ))
answer_twelve()

{'14', '32', '37', '45', '46'}

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [42]:
def answer_thirteen():
    
    return nx.Graph(answer_six().to_undirected())
answer_thirteen()

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [44]:
def answer_fourteen():
    G_un = answer_thirteen()
    transitivity = nx.transitivity(G_un)
    clustering = nx.average_clustering(G_un)
    return tuple((transitivity, clustering))
answer_fourteen()

(0.570111160700385, 0.6975272437231417)